In [48]:
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import time
import pandas as pd
import os
import dotenv

dotenv.load_dotenv()

True

In [3]:
openai_base_url = os.getenv("BASE_URL")
openai_api_key = os.getenv("API_KEY")
model = os.getenv("MODEL_NAME")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")

In [4]:
pc = Pinecone(api_key= pinecone_api_key)
client = OpenAI(
    api_key= openai_api_key,
    base_url= openai_base_url
)

## Try Embedding 

In [32]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_client = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"device":"cuda"},
    encode_kwargs={"normalize_embeddings": True}
    
)

def get_embedding(embedding_client, text_input): 
    embeddings = embedding_client.embed_query(text_input) 
    
    return embeddings

def get_embedding_docs(embedding_client, list_input): 
    embeddings = embedding_client.embed_documents(list_input) 
    
    return embeddings

In [7]:
output = get_embedding(embedding_client, "hello world")
print(output)

c:\Users\crab\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:407: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[0.015196124091744423, -0.02257067896425724, 0.00854706484824419, -0.07417059689760208, 0.0038364450447261333, 0.002713541965931654, -0.0312679298222065, 0.04463399574160576, 0.04405520483851433, -0.007871157489717007, -0.025200776755809784, -0.03336651623249054, 0.01442792546004057, 0.046538181602954865, 0.008555087260901928, -0.016145814210176468, 0.007405782584100962, -0.0190124474465847, -0.11472628265619278, -0.01815764233469963, 0.12635932862758636, 0.029702913016080856, 0.025280991569161415, -0.03421788662672043, -0.04099962115287781, 0.006617289502173662, 0.010270639322698116, 0.0223622415214777, 0.0044363136403262615, -0.1273096650838852, -0.016149280592799187, -0.0203801691532135, 0.04721205681562424, 0.011579878628253937, 0.0681871771812439, 0.007298607844859362, -0.017853017896413803, 0.04078211635351181, -0.010269439779222012, 0.023757049813866615, 0.010602893307805061, -0.02858438715338707, 0.008159706369042397, -0.0151804955676198, 0.03089623898267746, -0.065979942679405

In [9]:
len(output)

384

## Wrangle Dataset

In [11]:
df = pd.read_json ("products/products.jsonl", lines= True)

In [12]:
df.head(5)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",13.0,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",9.5,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",14.0,4.8,Latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",7.5,4.6,chocolat_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],6.0,4.9,Espresso_shot.webp


In [14]:
df['text'] = df['name'] + '  :  ' + df['description'] +\
    '  --  Ingredients: ' + df['ingredients'].astype(str) +\
        " -- Price: " + df['price'].astype(str) +\
            " -- Rating: " + df['rating'].astype(str)

In [15]:
df['text'].head(2)

0    Cappuccino  :  A rich and creamy cappuccino ma...
1    Jumbo Savory Scone  :  Deliciously flaky and b...
Name: text, dtype: object

In [37]:
texts = df['text'].tolist()

In [38]:
print(texts)

["Cappuccino  :  A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations.  --  Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 13.0 -- Rating: 4.7", "Jumbo Savory Scone  :  Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea.  --  Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 9.5 -- Rating: 4.3", "Latte  :  Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break.  --  Ingredient

In [39]:
with open('products/sama_sama_coffee_about_us.txt', encoding='utf-8') as f:
    sama_sama_coffee_about_section = f.read()

about_section_full_text  = "Coffee shop Sama Sama Coffee about section:  " + sama_sama_coffee_about_section

texts.append(about_section_full_text) 
    

In [40]:
print(texts)

["Cappuccino  :  A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations.  --  Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 13.0 -- Rating: 4.7", "Jumbo Savory Scone  :  Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea.  --  Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 9.5 -- Rating: 4.3", "Latte  :  Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break.  --  Ingredient

In [41]:
with open('products/menu_items_text.txt', encoding='utf-8') as f:
    menu_item_text = f.read()

menu_item_text = "Menu items :  " + menu_item_text

texts.append(menu_item_text)
    

In [42]:
print(texts)

["Cappuccino  :  A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations.  --  Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 13.0 -- Rating: 4.7", "Jumbo Savory Scone  :  Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea.  --  Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 9.5 -- Rating: 4.3", "Latte  :  Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break.  --  Ingredient

## Generate embeddings

In [44]:
output = get_embedding_docs(embedding_client, texts)

In [45]:
print(output)

[[-0.012832550331950188, -0.06698798388242722, 0.004235239699482918, 0.004728049971163273, 0.03029412217438221, -0.04715515673160553, 0.03652333468198776, 0.014083119109272957, -0.024569448083639145, -0.04898747801780701, -0.048234324902296066, -0.032378874719142914, -0.002012608340010047, -0.04013702645897865, 0.03627590835094452, -0.016455696895718575, -0.022429313510656357, -0.08356252312660217, -0.07491197437047958, -0.004258876666426659, 0.05230478197336197, -0.0663536936044693, -0.10556355863809586, -0.0415819026529789, 0.036544911563396454, -0.048001740127801895, 0.08525780588388443, -0.021832827478647232, 0.0008626297931186855, -0.13355545699596405, -0.009722407907247543, 0.006568307522684336, -0.023000182583928108, -0.027604518458247185, -0.03390510752797127, -0.001292622764594853, 0.06662176549434662, -0.07985130697488785, 0.0541062131524086, -0.0004590749158523977, -0.004105786327272654, 0.031212184578180313, -0.06324539333581924, -0.0030942047014832497, -0.00737563380971550

In [46]:
len(output)

20

## Push item to vector database

In [47]:
pc.create_index(
    name = index_name,
    dimension = 384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1",
    )
)

{
    "name": "coffeeshop",
    "metric": "cosine",
    "host": "coffeeshop-muw7mur.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [51]:
# wait for the index to finish
while not pc.describe_index(index_name).status.ready:
    time.sleep(1)
    
index = pc.Index(index_name)

vectors = []
for text,e in zip (texts,output):
    entry_id = text.split(":")[0]
    vectors.append(
        {"id" : entry_id,
         "values": e,
         "metadata": {"text":text}
         
         }
        
    )
    
index.upsert(vectors=vectors,
             namespace='ns1')

{'upserted_count': 20}

## Get closest document

In [52]:
embedding = get_embedding(embedding_client, text_input="is cappucino lactose free")
 

In [54]:
results = index.query(
    namespace='ns1',
    vector=embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

In [55]:
print(results)

{'matches': [{'id': 'Cappuccino  ',
              'metadata': {'text': 'Cappuccino  :  A rich and creamy '
                                   'cappuccino made with freshly brewed '
                                   'espresso, steamed milk, and a frothy milk '
                                   'cap. This delightful drink offers a '
                                   'perfect balance of bold coffee flavor and '
                                   'smooth milk, making it an ideal companion '
                                   'for relaxing mornings or lively '
                                   'conversations.  --  Ingredients: '
                                   "['Espresso', 'Steamed Milk', 'Milk Foam'] "
                                   '-- Price: 13.0 -- Rating: 4.7'},
              'score': 0.672838688,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup  ',
              'metadata': {'text': 'Sugar Free Vanilla syrup  :  Enjoy the '
                       